## part 1


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("official/mnist/dataset/", one_hot=True)
                                  

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting official/mnist/dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting official/mnist/dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting official/mnist/dataset/t10k-images-idx3-ubyte.gz
Extracting official/mnist/dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [6]:
mnist.train.num_examples

55000

In [7]:
mnist.test.num_examples

10000

## Preperation for Building CNN Model: Define Supporting Functions

### Initialize Weights in Filter

In [8]:
def intialize_weights (filter_shape):
    init_random_dist = tf.random.truncated_normal(filter_shape, stddev=0.1)
    return ( tf.Variable(init_random_dist) )

### Initialize bias

In [9]:
def initialize_bias(bias_shape):
    init_bias_vals = tf.constant(0.1, shape=bias_shape)
    return tf.Variable(init_bias_vals)

### Set Up Convolutional Layer and Perform Convulution: Dot Product

In [10]:
def create_convolution_layer_and_compute_dot_product(inputs, filter_shape):
    filter_initialized_with_weights = intialize_weights(filter_shape)
    conv_layer_outputs = tf.nn.conv2d(inputs, filter_initialized_with_weights, strides=[1, 1, 1, 1], padding='SAME')
    return (conv_layer_outputs)


### Set Up Convolutional Layer and Perform Convukution: Dot Product

In [11]:
def create_relu_layer_and_compute_dotproduct_plus_b(inputs, filter_shape):
    b = initialize_bias ([filter_shape[3]])
    relu_layer_outputs = tf.nn.relu(inputs +b)
    return(relu_layer_outputs)

### Set Up Pooling Layer and Reduce Spatial Size

In [12]:
def create_maxpool2by2_and_reduce_spatial_size(inputs):
    pooling_layer_outputs = tf.nn.max_pool2d(inputs, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    return pooling_layer_outputs

### Set Up Fully Connected Layer and Perform Computation: (inputs*Weights)+ Bias

In [13]:
def create_fully_connected_layer_and_compute_dotproduct_plus_bias (inputs, output_size):
    input_size = int(inputs.get_shape()[1])
    W = intialize_weights ([input_size, output_size])
    b = initialize_bias ([output_size])
    fc_xW_plus_bias_outputs = tf.matmul(inputs, W) +b
    return (fc_xW_plus_bias_outputs)

## Pahse 1: Build The Convolution Neural Network

### Create Placeholder for inputs and Labels: x&y_true

In [14]:
x = tf.compat.v1.placeholder(tf.float32, shape=[None, 784])

In [15]:
y_true = tf.compat.v1.placeholder(tf.float32,[None, 10])

### Reshape the input placeholder x

In [16]:
x_image = tf.reshape(x, [-1,28,28,1])

### Create 1st Convolution Layer, ReLU layer and perform Computation: x*W+b

In [17]:
conv_layer_1_outputs \
    = create_convolution_layer_and_compute_dot_product (x_image, filter_shape=[5,5,1,32])
conv_relu_layer_1_outputs \
    = create_relu_layer_and_compute_dotproduct_plus_b (conv_layer_1_outputs, filter_shape=[5,5,1,32])

### Create 1st Pooling Layer and Reduce Spatial Size

In [18]:
pooling_layer_1_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_1_outputs)

### Create 2nd Convolutional layer, ReLU Layer and Perform Computation: x*W+b

In [19]:
conv_layer_2_outputs \
    = create_convolution_layer_and_compute_dot_product (pooling_layer_1_outputs, filter_shape=[5,5,32,64])
conv_relu_layer_2_outputs \
    = create_relu_layer_and_compute_dotproduct_plus_b (conv_layer_2_outputs, filter_shape=[5,5,32,64])

### Create 2nd Pooling Layer and Reduce Spatial Size

In [20]:
pooling_layer_2_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_2_outputs)

### Reshape/Flatten Data Making it Ready to be Fed inti 1st FC Layer

In [21]:
pooling_layer_2_outputs_flat = tf.reshape(pooling_layer_2_outputs, [-1,7*7*64])

In [22]:
fc_layer_1_outputs \
    = create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat, output_size=1024)
fc_relu_layer_1_outputs = tf.nn.relu(fc_layer_1_outputs)


In [23]:
hold_prob = tf.compat.v1.placeholder(tf.float32)
rate = 1 - hold_prob
fc_dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, rate=rate)


In [24]:
y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

In [25]:
softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred)
cross_entropy_mean = tf.reduce_mean (softmax_cross_entropy_loss)


In [26]:
optimizer = tf.compat.v1.train.AdamOptimizer (learning_rate=0.001)

In [27]:
cnn_trainer = optimizer.minimize(cross_entropy_mean)

In [28]:
vars_initializer = tf.compat.v1.global_variables_initializer()

In [29]:
steps = 5000

In [31]:
with tf.compat.v1.Session() as sess:
    sess.run(vars_initializer)
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob:0.5})
        if i%100 == 0:
            print('ON STEP :{}'.format(i))
            print('ACCURACY: ')
            
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            test_accuracy = sess.run ( acc, feed_dict = {x:mnist.test.images, \
                                                         y_true: mnist.test.labels, \
                                                         hold_prob: 1.0} )
            print(test_accuracy)
            print('\n')
     
     
                                                         
                
        

OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 1 packages x 4 cores/pkg x 2 threads/core (4 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 core 0 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 4 maps to package 0 core 0 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 1 maps to package 0 core 1 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 5 maps to package 0 core 1 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 2 maps to package 0 core 2 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 6 maps to package 0 core 2 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 3 maps to package 0 core 3 thread 0 
OMP: Info #171: KMP_AFFINITY: O

ON STEP :0
ACCURACY: 
0.1662


ON STEP :100
ACCURACY: 
0.9402


ON STEP :200
ACCURACY: 
0.9614


ON STEP :300
ACCURACY: 
0.9681


ON STEP :400
ACCURACY: 
0.9726


ON STEP :500
ACCURACY: 
0.9768


ON STEP :600
ACCURACY: 
0.9786


ON STEP :700
ACCURACY: 
0.977


ON STEP :800
ACCURACY: 
0.9791


ON STEP :900
ACCURACY: 
0.9791


ON STEP :1000
ACCURACY: 
0.9837


ON STEP :1100
ACCURACY: 
0.9831


ON STEP :1200
ACCURACY: 
0.9849


ON STEP :1300
ACCURACY: 
0.9841


ON STEP :1400
ACCURACY: 
0.9851


ON STEP :1500
ACCURACY: 
0.9859


ON STEP :1600
ACCURACY: 
0.9846


ON STEP :1700
ACCURACY: 
0.983


ON STEP :1800
ACCURACY: 
0.9849


ON STEP :1900
ACCURACY: 
0.9868


ON STEP :2000
ACCURACY: 
0.984


ON STEP :2100
ACCURACY: 
0.9866


ON STEP :2200
ACCURACY: 
0.9872


ON STEP :2300
ACCURACY: 
0.9886


ON STEP :2400
ACCURACY: 
0.9865


ON STEP :2500
ACCURACY: 
0.9885


ON STEP :2600
ACCURACY: 
0.9865


ON STEP :2700
ACCURACY: 
0.9862


ON STEP :2800
ACCURACY: 
0.9875


ON STEP :2900
ACCURACY: 
0.98